In [2]:
# from nansat import Nansat
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import scipy
import nansat
# import sklearn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import PCA

import matplotlib as mpl
mpl.use('Qt5Agg')
import matplotlib.pyplot as plt

In [3]:
s2_path = r'C:/Users/ariane/shared/NERSC/classification/Subset_S2_20210716_resampled.tif'
rpoints = 'C:/Users/ariane/shared/NERSC/classification/rasterpoints.shp'
path = 'C:/Users/ariane/shared/NERSC/classification/'
# s2 = Nansat(path)
# print(s2)
# Gives Warning, but displays all Bands

In [4]:
points = gpd.read_file(rpoints)
class_names = points['classtxt'].unique()
print(class_names)
points

['Water' 'Seabed' 'Road' 'Pioneer' 'LSM' 'USM' 'Agriculture']


,classtxt,classid,B2,B3,B4,B8,B11,B12,NDVI,SAVI,MSAVI,NDMI,EVI,AWEI,NDWI,geometry
0,Water,1,244.0,376.0,252.0,148.0,172.0,147.0,-0.260000,2.758518,0.023151,-0.075000,-0.026450,0.037475,0.435115,POINT (8.78073 54.56462)
1,Water,1,231.0,410.0,279.0,126.0,152.0,124.0,-0.377778,2.746901,0.018096,-0.093525,-0.037994,0.065950,0.529851,POINT (8.79207 54.56766)
2,Water,1,262.0,402.0,236.0,119.0,154.0,127.0,-0.329577,2.779272,0.017793,-0.128205,-0.030564,0.061300,0.543186,POINT (8.77722 54.56884)
3,Water,1,205.0,339.0,210.0,105.0,140.0,128.0,-0.333333,2.802446,0.015666,-0.142857,-0.026711,0.041775,0.527027,POINT (8.79249 54.57158)
4,Water,1,308.0,489.0,330.0,162.0,166.0,159.0,-0.341463,2.700655,0.023948,-0.012195,-0.042718,0.081425,0.502304,POINT (8.78216 54.57501)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Agriculture,7,130.0,522.0,138.0,5644.0,2242.0,992.0,0.952266,1.901873,0.977237,0.431397,0.888236,-1.101900,-0.830684,POINT (8.85396 54.65587)
272,Agriculture,7,355.0,886.0,728.0,3010.0,1737.0,898.0,0.610487,1.977798,0.560582,0.268169,0.387686,-0.662600,-0.545175,POINT (8.86956 54.65963)
273,Agriculture,7,338.0,924.0,827.0,2730.0,1452.0,764.0,0.535001,1.975342,0.504292,0.305595,0.313881,-0.489550,-0.494253,POINT (8.86083 54.65956)
274,Agriculture,7,388.0,952.0,894.0,2830.0,1567.0,761.0,0.519871,1.941311,0.519487,0.287241,0.316671,-0.526025,-0.496563,POINT (8.86424 54.66112)


In [5]:
with rasterio.open(s2_path) as s2: 
    meta = s2.meta
    arr = s2.read()

meta
arr

array([[[ 157,  183,  193, ...,   77,    1,    3],
        [ 201,  183,  182, ...,    1,    1,    1],
        [ 201,  190,  209, ...,    1,   19,    6],
        ...,
        [ 199,  170,  226, ...,  138,  167,  267],
        [ 187,  214,  227, ...,  152,  144,  157],
        [ 188,  223,  244, ...,  148,  141,  164]],

       [[ 335,  320,  355, ...,  243,  137,  116],
        [ 367,  334,  335, ...,  147,  112,  141],
        [ 414,  357,  380, ...,  152,  135,  148],
        ...,
        [ 382,  386,  403, ...,  512,  620,  713],
        [ 400,  396,  421, ...,  577,  598,  588],
        [ 390,  404,  438, ...,  565,  532,  526]],

       [[ 284,  273,  315, ...,  333,  101,  101],
        [ 299,  284,  297, ...,  140,   88,   72],
        [ 315,  311,  317, ...,  139,  114,  108],
        ...,
        [ 181,  182,  191, ...,  190,  253,  339],
        [ 183,  189,  213, ...,  173,  200,  240],
        [ 183,  205,  238, ...,  243,  258,  274]],

       [[1009, 1019,  930, ..., 2156,

In [6]:
points = points.to_crs(meta["crs"])
coord_list = [(x,y) for x,y in zip(points['geometry'].x , points['geometry'].y)]
with rasterio.open(s2_path) as s2: 
    samplevalues = [x for x in s2.sample(coord_list)]

In [7]:
# points.to_file('C:/Users/ariane/shared/NERSC/classification/rasterpoints.csv', driver="CSV")
cols = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'] # 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'AWEI', 'NDWI']
X_train, X_test, y_train, y_test=train_test_split(points[cols], points['classid'], train_size=0.7, random_state=42)
X_train
# X_train.corr()
# pc = PCA().fit(X_train)
# pc_train = pc.transform(X_train)
# pc_test = pc.transform(X_test)


,B2,B3,B4,B8,B11,B12
38,597.0,895.0,874.0,1188.0,979.0,665.0
259,218.0,589.0,475.0,2964.0,2601.0,1502.0
112,549.0,778.0,684.0,2610.0,2257.0,1266.0
57,358.0,518.0,594.0,806.0,295.0,172.0
97,519.0,872.0,701.0,3946.0,1918.0,1043.0
...,...,...,...,...,...,...
188,353.0,786.0,412.0,4820.0,1684.0,802.0
71,459.0,691.0,673.0,1124.0,677.0,443.0
106,695.0,810.0,808.0,2558.0,2055.0,1168.0
270,97.0,573.0,174.0,4708.0,2115.0,957.0


In [8]:
clf = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=50)
# clf.fit(pc_train, y_train)
clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=50)

In [9]:
clf_classified = clf.predict(X_test)
#clf_classified_pca = clf.predict(pc_test)
# clf_classified.shape

In [10]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
# scores = cross_val_score(clf, pc_train, y_train, cv=5)
scores

array([0.84615385, 0.82051282, 0.92307692, 0.92105263, 0.94736842])

In [11]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# user warning: X does not have valid feature names, but RFC was fitted with feature names --> ? 

# classification = clf_classified.reshape((rows, cols))

print(clf_classified)

# output = 'C:/Users/ariane/shared/NERSC/classification/outputRFC_w.tif'

0.89 accuracy with a standard deviation of 0.05
[1 5 6 5 6 2 5 6 5 5 2 5 2 7 5 2 7 5 3 7 2 2 5 5 7 6 3 2 1 2 1 3 1 5 5 2 6
 2 1 2 2 4 2 1 3 6 1 6 5 6 3 5 5 1 5 1 7 5 1 5 5 3 2 4 1 6 7 2 5 3 3 6 7 1
 1 3 6 3 2 3 4 6 7]


In [ ]:
# np.save('/home/ariane/nansen/classification/better_classification.npy', classification)
# classification = np.load('/home/ariane/nansen/classification/better_classification.npy')


#build confusion matrix (x-axis: predicted pixels, y-axis: reference pixels; class 1,2,3,4,5)
cm = confusion_matrix(y_test, clf_classified, labels = clf.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = clf.classes_)
disp.plot()
# %matplotlib inline
plt.show()
# driver = gdal.GetDriverByName('GTiff')
# print(driver)
# rows, cols = classification.shape
# print(rows, cols)
# rasterDS = driver.Create(output, cols, rows)
# print(rasterDS)

# rasterDS.SetGeoTransform(geo_transform)
# rasterDS.SetProjection(projection)
# rasterDS.GetRasterBand(1).WriteArray(classification)
# rasterDS.GetRasterBand(1).SetNoDataValue(0)
# rasterDS.FlushCache()

In [ ]:
# plot_confusion_matrix(clf, pc_test, y_test)

# cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                              display_labels=clf.classes_)

from sklearn.metrics import classification_report

In [ ]:
#calculate number of pixels that were classified correctly
accuracy_score(y_test, clf_classified, normalize=False)

pd.DataFrame(classification_report(y_test, clf_classified, digits=2, output_dict=True))

In [15]:
# accuracy_score(y_test, clf_classified, normalize=True)

In [16]:
# calculate precision score per class
# precision_score(y_test, clf_classified, average=None)

In [17]:
# calculate recall score per class
# recall_score(y_test, clf_classified, average=None)

In [18]:
# arr
#take the (multi layer) raster (arr) and flatten into 1-dimensional array
# arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])] 

In [20]:
### Save the model
import joblib

joblib.dump(clf, path + 'random_forest.joblib')
loaded_clf = joblib.load(path+'random_forest.joblib')
# loaded_clf.predict(pc_test)
loaded_clf.predict(X_test)

array([1, 5, 6, 5, 6, 2, 5, 6, 5, 5, 2, 5, 2, 7, 5, 2, 7, 5, 3, 7, 2, 2,
       5, 5, 7, 6, 3, 2, 1, 2, 1, 3, 1, 5, 5, 2, 6, 2, 1, 2, 2, 4, 2, 1,
       3, 6, 1, 6, 5, 6, 3, 5, 5, 1, 5, 1, 7, 5, 1, 5, 5, 3, 2, 4, 1, 6,
       7, 2, 5, 3, 3, 6, 7, 1, 1, 3, 6, 3, 2, 3, 4, 6, 7], dtype=int64)

In [21]:
clf_classified

array([1, 5, 6, 5, 6, 2, 5, 6, 5, 5, 2, 5, 2, 7, 5, 2, 7, 5, 3, 7, 2, 2,
       5, 5, 7, 6, 3, 2, 1, 2, 1, 3, 1, 5, 5, 2, 6, 2, 1, 2, 2, 4, 2, 1,
       3, 6, 1, 6, 5, 6, 3, 5, 5, 1, 5, 1, 7, 5, 1, 5, 5, 3, 2, 4, 1, 6,
       7, 2, 5, 3, 3, 6, 7, 1, 1, 3, 6, 3, 2, 3, 4, 6, 7], dtype=int64)

In [22]:
meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 960,
 'height': 1091,
 'count': 6,
 'crs': CRS.from_epsg(32632),
 'transform': Affine(10.0, 0.0, 484550.0,
        0.0, -10.0, 6057160.0)}

In [23]:
#take the (multi layer) raster (arr) and flatten into 1-dimensional array
arrays_flat = [arr[i, :, :].flatten() for i in range(arr.shape[0])] 

meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 960,
 'height': 1091,
 'count': 6,
 'crs': CRS.from_epsg(32632),
 'transform': Affine(10.0, 0.0, 484550.0,
        0.0, -10.0, 6057160.0)}

In [24]:
#define which band refers to which column in the raster data frame
#the order (0-X) is the same as in the original raster as it can be seen in QGIS UI
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})

In [25]:
cols_raster_df = raster_df.columns
#show length of raster data frame to check number
len(raster_df.columns)
#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df

,B2,B3,B4,B8,B11,B12
0,157,335,284,1009,445,271
1,183,320,273,1019,373,229
2,193,355,315,930,373,229
3,229,421,375,873,340,208
4,195,379,350,846,340,208
...,...,...,...,...,...,...
1047355,138,456,200,5112,1436,635
1047356,165,507,230,4924,1436,635
1047357,148,565,243,4804,1250,584
1047358,141,532,258,4308,1250,584


In [26]:
raster_df.columns

Index(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], dtype='object')

In [27]:
#classify the raster
raster_pred = clf.predict(raster_df)

In [28]:
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)

(1091, 960)


In [30]:
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_July_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out

(1091, 960)


array([[5, 5, 5, ..., 5, 7, 5],
       [5, 5, 5, ..., 7, 7, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [1, 1, 1, ..., 7, 7, 6],
       [1, 1, 1, ..., 7, 7, 7],
       [1, 1, 1, ..., 5, 5, 5]], dtype=int64)

In [1]:
### Apply to more sentinel 2 scenes

path = 'C:/Users/ariane/shared/NERSC/classification/'

with rasterio.open(path+'Subset_S2A_20210909_resampled.tif') as s2: 
    samplevalues = [x for x in s2.sample(coord_list)]

keys = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
arrays_flat = {keys[i]:arr[i, :, :].flatten() for i in range(arr.shape[0])} 
meta
raster_df = pd.DataFrame({
    "B2" : arrays_flat[0],
    "B3" : arrays_flat[1],
    "B4" : arrays_flat[2],
    "B8" : arrays_flat[3],
    "B11" : arrays_flat[4],
    "B12" : arrays_flat[5]})
cols_raster_df = raster_df.columns
#show length of raster data frame to check number
len(raster_df.columns)
#show if it worked and all the column-names are correct and in the correct order in the raster data frame
raster_df
raster_df.columns
#classify the raster
raster_pred = clf.predict(raster_df)
#reshape from array to raster
#take shape from original raster
shape_out = (arr.shape[1], arr.shape[2])
print(shape_out)
#show shape from original raster: 2-dimensional (correct? multidimensional?)
shape_out

#final calssified raster is the classified raster but reshaped in the shape as the original --> multidimensional
raster_out = raster_pred.reshape(shape_out)
print(raster_out.shape)

### Output path
output = path + 'RF_Sep_Classification.tif'
output
#define output data type (same as original)
meta["dtype"] = 'float32'
meta["count"] = 1

#test if it worked
meta
with rasterio.open(output, 'w', **meta) as dst:
    dst.write(raster_out, 1)
    
raster_out


NameError: name 'rasterio' is not defined